In [1]:
import json
import re
import sys
import os
import pandas as pd
import socket
import urllib
import win32con
import win32gui
import requests
import datetime
import struct
import time
import smtplib as smtp
import base64
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook, tqdm
from random import randint
from time import sleep
from win32api import GetModuleHandle
pd.set_option('display.max_columns', None)

In [3]:
def get_r(url_page, id_auc):
    flag = True
    while flag:
        try:
            r = requests.post(url_page, data={'id': id_auc})
            flag = False
        except socket.timeout as e:
            print(type(e))
            flag = True
    return r
def pretty_print(r):
    for offer in r.json()['offerInfo']:
        print(list(offer.values())[0]['number'], ' | ',
              list(offer.values())[0]['code'], ' | ',
              list(offer.values())[0]['bestOfferFormated'], ' | ', 
              list(offer.values())[0]['bestOfferDateTime'])
    print('\n')
def get_numbers_codes(r):
    numbers = []
    codes = []
    for offer in r.json()['offerInfo']:
        numbers.append(int(list(offer.values())[0]['number']))
        codes.append(list(offer.values())[0]['code'])
    return numbers, codes, sum([x == y for x, y in zip(numbers, codes)])
def write_history(r):
    time_mark = str(datetime.datetime.now()).replace(':', '_').replace('.', '_')
    with open('history_{}.txt'.format(time_mark), 'w') as outfile:
        json.dump(r.json(), outfile)
def send_mail(email_text):
    email = '***@***.ru'
    password = base64.b64decode(b'**********').decode('utf-8')
    dest_email = ['***@***.ru', '***@***.ru']
    subject = 'Auction warning'
    message = 'From: {}\nTo: {}\nSubject: {}\n\n{}'.format(email, ', '.join(dest_email), subject, email_text)
    server = smtp.SMTP_SSL('smtp.yandex.com')
    #server.set_debuglevel(1)
    #server.ehlo(email)
    server.login(email, password)
    server.auth_plain()
    server.sendmail(email, dest_email, message)
    server.quit()

In [4]:
def main():
    os.chdir('C:/Users/vgarshin/Documents/Py/etorgi')
    cur_dir = 'history'
    if not os.path.exists(cur_dir):
        os.makedirs(cur_dir)
    os.chdir('C:/Users/vgarshin/Documents/Py/etorgi' + '/' + cur_dir)
    url_page = 'https://www.etp-ets.ru/data/get-auction-info'
    id_auc = 359169
    timeout = 5
    rounds = 2
    history = []
    email_text = 'VTB is second for {} rounds, or {} seconds'.format(rounds, timeout * rounds)
    for _ in range(rounds):
        r = get_r(url_page, id_auc)
        pretty_print(r)
        write_history(r)
        history.append(r)
        sleep(timeout)
    print('end of init history\n')
    while True:
        try:
            r = get_r(url_page, id_auc)
            pretty_print(r)
            write_history(r)
            history.append(r)
            history = history[-rounds:]
            battle = []
            for hel in history:
                for offer in hel.json()['offerInfo']:
                    battle.append([list(offer.values())[0]['number'],
                                   list(offer.values())[0]['code']])
            if all([x[1] == 2 for x in battle if x[0] == '2']) == True:
                send_mail(email_text)
            sleep(timeout)
        except KeyboardInterrupt:
            print('KeyboardInterrupt')
            break

In [ ]:
if __name__ == '__main__':
    main()